# Decision Theory Project - TicTacToe
*By Jelle Huibregtse and Aron Hemmes*

Below is a TicTacToe environment build from scratch with an Agent based on reward.

# 1. Setup
## Loading in libraries

In [1]:
# Libraries
import random
import functools
from enum import Enum
from IPython.display import display
from ipywidgets import Layout, Button, HTML, Box
from typing import List, Tuple
from copy import copy

# Layout
field_layout = Layout(width="50px", height="50px")
wide_layout = Layout(width="158px")
column_layout = Layout(flex_flow="column")

# Formatting
%load_ext nb_black

<IPython.core.display.Javascript object>

## Configuring layout

In [2]:
%%HTML
<style>
.widget-button {
    outline: none !important;
}

.widget-html-content {
    white-space: pre-wrap;
    line-height: normal !important;
}
</style>

<IPython.core.display.Javascript object>

# 2. Definition of the Environment

The code below defines all characteristics of our tic-tac-toe environment with the following characteristics:

Environment state:

- the player type is either X or O
- the opposing player (agent) is either X or O depending on the player
- X and O take turns placing an X or O on empty fields untill either one has won or there are no more fields left on the board
- a board starts out empty and can contain X and O marks

A TicTacToeEnvironment object has the following methods:
- `reset()` which completely resets the board to an empty state.
- `update()` Updates the visualisation of the current TicTacToe game.
- `render()` Visualisation of the current TicTacToe game.
- `change_player()` The player switches between X and O and resets the board.
- `field_click(field)` The player sets a field to a itself if the field is None.

Aditionally to allow an agent to calculate optimal decisions using model information, these methods are also available:
- `get_turns()` Returns the amount of turns that have passed.
- `get_active()` Returns which player's turn it currently is.
- `get_state()` Returns a string representing the current board.
- `get_reward(field)` Simplified version $R(s,a)$ of the general reward function: $R(s,â) = 1$, $R(s,ã) = -1$, $R(s,a) = 0$
- `get_winnable_fields(player)` Returns all fields for a player that allow them to win. 
- `get_result()` Returns which player's got 3 in a row, otherwise returns None.
- `set_field(field)` Sets a field to currently active player.
- `step()` Processes the agent policies and returns the action, state and reward for the current active agent policy.

In [3]:
rows = [
    [int(a), int(b), int(c)]
    for a, b, c in ["036", "012", "147", "345", "048", "246", "678", "258"]
]


class Type(Enum):
    X = 1
    O = 2


class TicTacToeEnvironment:
    def __init__(self, policies: list = None):
        self.board = [None for _ in range(9)]
        self.player = Type.X
        self.policies = policies

        if not policies == None and not len(policies) > 1:
            self.step()

    def get_turns(self) -> int:
        return len([field for field in self.board if not field == None])

    def get_active(self) -> Type:
        return Type.X if self.get_turns() % 2 == 0 else Type.O

    def get_state(self) -> str:
        return str([f.name if not f == None else "E" for f in self.board]).replace(
            "'", ""
        )

    def get_result(self) -> Type:
        # Check for three of the same marks in a row
        board = self.board
        for row in rows:
            # If not none and three in a row are the same
            if (
                not board[row[0]] == None
                and board[row[0]] == board[row[1]] == board[row[2]]
            ):
                # Return board
                return board[row[0]]

    def step(self) -> Tuple[Type, int, float, str]:
        # Execute agent code
        field = None
        if self.get_result() == None:
            if not self.policies == None:
                if len(self.policies) == 1:
                    if not self.player == self.get_active():
                        field = self.policies[0](self)
                if len(self.policies) == 2:
                    field = self.policies[0 if self.player == self.get_active() else 1](
                        self
                    )

        if not field == None:
            player, reward = self.get_active(), self.get_reward(field)
            self.set_field(field)
            return player, field, reward, self.get_state()

    def get_reward(self, field: int) -> float:
        player_fields = self.get_winnable_fields(self.get_active())
        opposing_fields = self.get_winnable_fields(
            Type.O if self.get_active() == Type.X else Type.X
        )
        # If player places in field with 3 in a row, return 1.0
        if field in player_fields:
            return 1.0
        # If player doesn't place in field where opposing can get 3 in a row, return -1.0
        elif len(opposing_fields) > 0 and not field in opposing_fields:
            return -1.0
        # Return 0.0 by default
        return 0.0

    def reset(self) -> None:
        self.board = [None for _ in range(9)]
        self.update()

        if not self.policies == None and not len(self.policies) > 1:
            self.step()

    def set_field(self, field: int) -> None:
        # Set field to current
        self.board[field] = self.get_active()

        # Update the board
        self.update()

    def get_winnable_fields(self, player: Type) -> List[int]:
        # If there are 2 matching player type and 1 matching None return the fields
        result = []
        for row in rows:
            fields = [[c, self.board[c]] for c in row]
            noneMatches = [f for f in fields if f[1] == None]
            playerMatches = [f for f in fields if f[1] == player]
            if len(noneMatches) == 1 and len(playerMatches) == 2:
                result.append(noneMatches[0][0])

        return result

    def change_player(self) -> None:
        self.player = Type.O if self.player == Type.X else Type.X
        self.reset()

    # ----------------------------
    # Visual board methods
    # ----------------------------
    def field_click(self, field: int) -> None:
        if self.policies == None or not len(self.policies) > 1:
            if self.get_result() == None and self.board[field] == None:
                if self.policies == None or self.get_active() == self.player:
                    self.set_field(field)
                if not self.policies == None and not self.get_active() == self.player:
                    self.step()

    def update(self) -> None:
        if hasattr(self, "field_buttons"):
            for i in range(len(self.field_buttons)):
                self.field_buttons[i].description = (
                    self.board[i].name if not self.board[i] == None else " "
                )
            if hasattr(self, "player_select_button"):
                self.player_select_button.description = "PLAYER " + self.player.name
            result = self.get_result()
            if not result == None:
                self.result_text.value = "winner is <b>{}</b>".format(result.name)
            else:
                self.result_text.value = ""

    def render(self) -> None:
        if not hasattr(self, "field_buttons"):
            self.field_buttons = []
            elements = []

            # Add header
            if not self.policies == None and not len(self.policies) > 1:
                elements.append(HTML(value="<h1>TicTacToe</h1>"))

            # Add field buttons
            buttons = []
            rows = []
            for i in range(9):
                btn = Button(layout=field_layout)
                btn.on_click(functools.partial(self.field_click, i))
                buttons.append(btn)
                self.field_buttons.append(btn)
                if (i + 1) % 3 == 0:
                    rows.append(Box(buttons))
                    buttons = []
            elements.append(Box(children=rows, layout=column_layout))

            # Add player select
            if not self.policies == None and not len(self.policies) > 1:
                player_btn = Button(
                    description="PLAYER " + self.player.name, layout=wide_layout
                )
                player_btn.on_click(self.change_player)
                self.player_select_button = player_btn

                elements.append(player_btn)

            # Add reset button
            if not self.policies == None and not len(self.policies) > 1:
                reset_btn = Button(description="RESET", layout=wide_layout)
                reset_btn.on_click(self.reset)
                elements.append(reset_btn)

            # Add result text
            result_text = HTML()
            elements.append(result_text)
            self.result_text = result_text

            # Display elements and data
            display(Box(children=elements, layout=column_layout))

        # Update the board
        self.update()

<IPython.core.display.Javascript object>

## Creation of an Environment
The `TicTacToeEnvironment` class allows the creation of an tic-tac-toe environment that can take in different policies as parameters. An example is below with no agent (no policy) and player vs. player. Note that game is interactable. Furthermore, this is the default state of the environment.

In [4]:
# Initializing environment and rendering
tictactoe = TicTacToeEnvironment()
tictactoe.render()

Box(children=(Box(children=(Box(children=(Button(layout=Layout(height='50px', width='50px'), style=ButtonStyle…

<IPython.core.display.Javascript object>

## Action Space

The actions space of the tic-tac-toe environment is placing a `Type` on one of the nine fields. The `Type` enum is as follows:

In [5]:
for nr, action in enumerate(Type, 1):
    print("action", nr, "is", action)

action 1 is Type.X
action 2 is Type.O


<IPython.core.display.Javascript object>

## Transitions
Usually, we also define a transition function $T(s,a,s')$ that gives the probability of moving from a state $s$ to $s'$ when performing action $a$. This is only used in an environment where we are unsure or where it is unclear if some action always gives a predictable outcome. However, since tic-tac-toe is a simple game where every action is defined, we won't be needing a transition function.

# 3. Unbeatable computer

An unbeatable computer that consistently plays the same best moves, sometimes there are multiple equally good moves, then it takes the first one of these moves.

In [6]:
def computed_best_policy(tictactoe: TicTacToeEnvironment) -> int:
    """Return field computed with if statements"""
    player = tictactoe.get_active()
    opponent = Type.X if player == Type.O else Type.O

    # If player is able to make 3 in a row, place to win
    fields = tictactoe.get_winnable_fields(player)
    if len(fields) > 0:
        return fields[0]

    # If opponent is able to make 3 in a row, place to prevent
    fields = tictactoe.get_winnable_fields(opponent)
    if len(fields) > 0:
        return fields[0]

    # If center is empty, place in center
    if tictactoe.board[4] == None:
        return 4

    # If a corner is empty and a corner on the opposite diagonal side is also empty, place in this first empty corner
    empty_diagonal_corners = [
        i[0]
        for i in [[0, 8], [2, 6]]
        if tictactoe.board[i[0]] == None and tictactoe.board[i[1]] == None
    ]
    if len(empty_diagonal_corners) > 0:
        return empty_diagonal_corners[0]

    # If a corner is empty, place in first empty corner
    empty_corners = [i for i in [0, 2, 6, 8] if tictactoe.board[i] == None]
    if len(empty_corners) > 0:
        return empty_corners[0]

    # Place in first empty field
    empty_fields = [
        [i, tictactoe.board[i]]
        for i in range(len(tictactoe.board))
        if tictactoe.board[i] == None
    ]
    if len(empty_fields) > 0:
        return empty_fields[0][0]


# Initializing environment and rendering
tictactoe = TicTacToeEnvironment([computed_best_policy])
tictactoe.render()

Box(children=(HTML(value='<h1>TicTacToe</h1>'), Box(children=(Box(children=(Button(layout=Layout(height='50px'…

<IPython.core.display.Javascript object>

# 4. Random Agent

Two agents play against each other: 

- Agent X: Uses the `computed_best_policy`.
- Agent O: Uses the `random_policy`.

In the cell below, you can see the effect of an agent with a random policy choosing an arbitrary action regardless of the new state, playing against the `computed_best_policy`.

In [7]:
def random_policy(tictactoe: TicTacToeEnvironment) -> int:
    """Return random field in empty fields"""

    # Getting all the empty fields
    empty_fields = [f for f in range(9) if tictactoe.board[f] == None]

    # Choose random empty field
    if len(empty_fields) > 0:
        return random.choice(empty_fields)


total_reward = 0.0

tictactoe = TicTacToeEnvironment([computed_best_policy, random_policy])
while (
    tictactoe.get_result() == None
    and len([f for f in tictactoe.board if f == None]) > 0
):
    player, field, reward, state = tictactoe.step()
    if player == Type.O:
        total_reward += reward
        print(
            "action: {}\tstate: {}, reward: {}".format(
                player.name + " " + str(field), state, reward
            )
        )
    else:
        print("action: {}\tstate: {}".format(player.name + " " + str(field), state))
print(
    "\nEpisode done after {} steps. Total reward: {}".format(
        tictactoe.get_turns(), total_reward
    )
)

action: X 4	state: [E, E, E, E, X, E, E, E, E]
action: O 0	state: [O, E, E, E, X, E, E, E, E], reward: 0.0
action: X 2	state: [O, E, X, E, X, E, E, E, E]
action: O 3	state: [O, E, X, O, X, E, E, E, E], reward: -1.0
action: X 6	state: [O, E, X, O, X, E, X, E, E]

Episode done after 5 steps. Total reward: -1.0


<IPython.core.display.Javascript object>

If you run the cell above a number of times, you can observe two things:
- The total reward and steps will vary from run to run.
- O never wins, but sometimes the game ends in a tie.

Each run from start state until stop state is called an episode.  
Let's assemble some statistics on the episodes of the random agent:

In [8]:
from statistics import mean, stdev


def run_one_episode(policy) -> int:
    tictactoe = TicTacToeEnvironment([computed_best_policy, policy])
    total_reward = 0.0
    while (
        tictactoe.get_result() == None
        and len([f for f in tictactoe.board if f == None]) > 0
    ):
        player, field, reward, state = tictactoe.step()
        if player == Type.O:
            total_reward += reward
    return total_reward


def measure_performance(policy, nr_episodes: int = 100):
    N = nr_episodes
    print("statistics over", N, "episodes")
    all_rewards = []
    for _ in range(N):
        episode_reward = run_one_episode(policy)
        all_rewards.append(episode_reward)

    print("mean: {:6.2f}, sigma: {:6.2f}".format(mean(all_rewards), stdev(all_rewards)))
    print()
    for n, episode_reward in enumerate(all_rewards[:5], 1):
        print("ep: {:2d}, total reward: {:5.2f}".format(n, episode_reward))
    print("......")
    for n, episode_reward in enumerate(all_rewards[-5:], len(all_rewards) - 5):
        print("ep: {:2d}, total reward: {:5.2f}".format(n, episode_reward))


measure_performance(random_policy)

statistics over 100 episodes
mean:  -0.96, sigma:   0.20

ep:  1, total reward: -1.00
ep:  2, total reward: -1.00
ep:  3, total reward: -1.00
ep:  4, total reward: -1.00
ep:  5, total reward: -1.00
......
ep: 95, total reward: -1.00
ep: 96, total reward: -1.00
ep: 97, total reward: -1.00
ep: 98, total reward: -1.00
ep: 99, total reward: -1.00


<IPython.core.display.Javascript object>

# 5. Decisions based on reward agent

Next we have the code for the agent that makes the decision based on reward. Since, we are working with a **Markov Decision Process** (MDP), we also have to define a reward function $R(s,a)$ that returns a value based on performing action $a$ in state $s$ both of which have been previously defined.

We will define the reward function $R(s,a)$ as follows:
- Trivially, if the agent performs some action $â$ that wins the game from $s$, then $R(s,â) = 1$.
- If the agent makes a mistake where the wrong action $ã$ loses the game, we then say $R(s,ã) = -1$.
- When nothing happens we can simply say $R(s,a)=0$.

We will be using Q-learning to find an optimal policy that the agent uses to decide which actions to pick. We will simply denote our policy as the action $a$ that maximizes a function $Q(s,a)$ when the agent is in some state $s$. So, we would have something like:

$$a^{best} = \text{arg}\max_{a\in A}Q(s, a)$$

For every state, each action has an associated value of $Q$ and we want to pick the $Q$ with the highest value. So, to compute $Q(s,a)$, the agent has to go over each possible pairs of states and actions while getting feedback from the reward function. We will update $Q(s,a)$ iteratively by letting the agent play. We will update $Q$ as follows:

$$Q(s,a)^{new} \leftarrow (1 - \alpha)\cdot Q(s,a)+\alpha\cdot(R(s,a)+\gamma\cdot\max_{â\in A}Q(ŝ, â))$$

- We perform an action $a$ in the current state $s$.
- $\max_{â\in A}Q(ŝ, â))$ takes into account future states and returns the largest $Q$, ŝ is the state that is the new state after performing $a$. Then $â$ is the best action.
- $\alpha$ is the learning rate that decides to what extent we overwrite the old value, we will use $\alpha=0.1$.
- The discount factor $\gamma$ decides how much future rewards should be weighted compared to present rewards at the current time step $t$. We will be using $\gamma=0.9$.

We will find (read: learn) the best values for $Q(s,a)$. This will be done by letting two agents play against each other. To make sure it is balanced and that agents also seek out new options we will introduce a probability $\epsilon$ that an agent picks a random action.

In [9]:
Q: List[Tuple[List[float], List[Type]]] = []


def get_max_q(current_state: List[Type]) -> float:
    """Returns the maximum Q value given a state and list of actions (input is hash keys)"""
    max_q = 0.0
    for reward, state in Q:
        if max(reward) > max_q:
            is_iteration = (
                len(
                    [
                        i
                        for i in range(len(current_state))
                        if current_state[i] == None or current_state[i] == state[i]
                    ]
                )
                == 9
            )
            if is_iteration:
                max_q = max(reward)
    return max_q


def update_q(
    tictactoe: TicTacToeEnvironment, field: int, 𝛼: float = 0.1, 𝛾: float = 0.9
) -> None:
    state = tictactoe.board

    reward = 0.5 + (tictactoe.get_reward(field) / 2)
    updated_state = [
        state[i] if not i == field else tictactoe.get_active()
        for i in range(len(state))
    ]
    max_q = get_max_q(updated_state)
    pos = [i for i in range(len(Q)) if Q[i][1] == state]
    pos = pos[0] if len(pos) > 0 else None
    if not pos == None:
        value = (1 - 𝛼) * Q[pos][0][field] + 𝛼 * (reward + 𝛾 * max_q)
        Q[pos][0][field] = value
    else:
        Q.append(([0.0 if i == field else 𝛼 * reward for i in range(9)], copy(state)))


def train_random_policy(tictactoe: TicTacToeEnvironment):
    available_fields = [
        i for i in range(len(tictactoe.board)) if tictactoe.board[i] == None
    ]

    field = random.choice(available_fields)

    update_q(tictactoe, field)

    return field


def train(nr_episodes: int = 5000):
    N = nr_episodes
    for _ in range(N):
        tictactoe = TicTacToeEnvironment([computed_best_policy, train_random_policy])
        while (
            tictactoe.get_result() == None
            and len([f for f in tictactoe.board if f == None]) > 0
        ):
            player, field, reward, state = tictactoe.step()


train()

<IPython.core.display.Javascript object>

In [10]:
def optimal_decision_policy(tictactoe: TicTacToeEnvironment) -> int:
    """Get best action given a set of possible actions in a given state"""

    actions = [i for i in range(len(tictactoe.board)) if tictactoe.board[i] == None]

    # Pick a random action at first
    best_action = random.choice(actions)

    # Find action with largest Q in given state
    max_q = 0
    q = [Q[i] for i in range(len(Q)) if Q[i][1] == tictactoe.board]
    q = q[0] if len(q) > 0 else None
    if not q == None:
        for i in range(len(q[0])):
            if q[0][i] > max_q:
                max_q = q[0][i]
                best_action = i

    return best_action


total_reward = 0.0

tictactoe = TicTacToeEnvironment([computed_best_policy, optimal_decision_policy])
while (
    tictactoe.get_result() == None
    and len([f for f in tictactoe.board if f == None]) > 0
):
    player, field, reward, state = tictactoe.step()
    if player == Type.O:
        total_reward += reward
        print(
            "action: {}\tstate: {}, reward: {}".format(
                player.name + " " + str(field), state, reward
            )
        )
    else:
        print("action: {}\tstate: {}".format(player.name + " " + str(field), state))

print(
    "\nEpisode done after {} steps. Total reward: {}".format(
        tictactoe.get_turns(), total_reward
    )
)

action: X 4	state: [E, E, E, E, X, E, E, E, E]
action: O 6	state: [E, E, E, E, X, E, O, E, E], reward: 0.0
action: X 0	state: [X, E, E, E, X, E, O, E, E]
action: O 8	state: [X, E, E, E, X, E, O, E, O], reward: 0.0
action: X 7	state: [X, E, E, E, X, E, O, X, O]
action: O 1	state: [X, O, E, E, X, E, O, X, O], reward: 0.0
action: X 2	state: [X, O, X, E, X, E, O, X, O]
action: O 3	state: [X, O, X, O, X, E, O, X, O], reward: 0.0
action: X 5	state: [X, O, X, O, X, X, O, X, O]

Episode done after 9 steps. Total reward: 0.0


<IPython.core.display.Javascript object>

Note that reward is always $0.0$, since the best computed policy and optimal decision policy are against each other. The best possible result for both is a tie.

# Value iteration vs Q-learning

## How does value iteration work
Let's dive a bit deeper into what value iteration is and the difference with Q-learning. For the value iteration reinforcement learning algorithm the agent knows two things before taking a specific action.

1. The probabilities of ending up in other new states given that specific action from the current state. Basically, the agent knows a **transition function**. So, we can formally define such a function as follows: A transition function $T(s,a,s')$ that gives the probability of moving from a state $s$ to $s'$ when performing action $a$.
2. The immediate reward that would be received for taking that action $a$ from the current state $s$ to $s'$. This is that reward function $R(s,a)$.

For value iteration we can say that the agent is able to look ahead and calculate which route to take based on transition and reward. The algorithm will take an expression known as an **experience tuple** to keep track of data that changes with each state and action. Since, value iteration is a model based approach, it builds a model of the transition function $T(s,a,s')$ and reward function $R(s,a)$ using the experience tuples. Then once the models are built we can use value iteration to determine the optimal values of each state. The optimal values of each state s are based on the action a that generates the best expected cumulative reward. So, we calculate the value of each state-action pair in the training phase, then at each time-step, we select the action where that state-action pair value is the highest ($max_aQ^*(s,a)$).

## The difference
So, in most cases it is not efficient to first build an complete model (again that means the transition and reward function) of the environment before you start. For two reasons: the model might take a long time to create and second, the environment might be unpredictable or always changing. You would want for the agent to able to adapt to that. So, Q-learning would be a model-free reinforcement learning algorithm. The agent has to interact with the environment and learn using trial-and-error. It cannot look ahead. In Q-learning the agent builds a table of **Q-values** denoted as $Q(s,a)$ as it takes an action $a$ from a state $s$ and the receives a reward $r$. The agent uses the table to see what action is best based on previous experiences (or outcomes) of taking some action.

## Policy iteration
Next, we also have policy iteration which is a way of finding the optimal policy given a set of states and actions. Let's assume we some policy $(\pi:S\rightarrow A)$ that assigns an action to each state. Action $\pi(s)$ will be chosen each time the system is a state s.

1. Evaluate a given policy (we can initialize a policy arbitrarily for all states ($s\in S$) by calculating a value function for all states $s\in S$ under a given policy. So the value function would be the expected reward collected at the first step, plus the expected discounted value at the next state. $$V_{\pi}(s) = E[R(s, \pi(s), s') + \gamma V(s')]$$.

2. Then we improve the policy by finding a better action for every state $s$ (we give each iteration a subscript, here 1). $$\pi_1(s)=\text{arg}\max_{a\in A}E[R(s, a, s') + \gamma V(s')]$$

3. We repeat these steps until we converge at an optimal value function.

# Sources
- For reinforcement learning (Q-learning): https://towardsdatascience.com/how-to-play-tic-tac-toe-using-reinforcement-learning-9604130e56f6
- Value iteration vs. Q-learning algorithms: https://automaticaddison.com/value-iteration-vs-q-learning-algorithm-in-python-step-by-step/
- Policy iteration: https://medium.com/@pesupavish/policy-iteration-easy-example-d3fd1eb98c6c